In [140]:
from pydantic import BaseModel, Field, PrivateAttr, create_model
from typing import List, Literal, Union

class ModifiedBaseModel(BaseModel):
    _is_wrapped = PrivateAttr(default=False)
      
    @classmethod
    def wrap_model(cls, 
                   with_reasoning: bool = True,
                   to_list: bool = True) -> BaseModel:

        original_name: str = cls.__name__
        prefix: str = "ListOf" if to_list else ""
        suffix: str = "WithReasoning" if with_reasoning else ""
        modified_name: str = f"{prefix}{original_name}{suffix}"
        args: dict = {"_is_wrapped": PrivateAttr(default=True),
                      "_original_name": PrivateAttr(default=original_name),
                      "__base__":ModifiedBaseModel}
        
        if with_reasoning:
            args['reasoning'] = (str, Field(description="Explain your actions here."))
        
        args[original_name.lower()] = (
            List[cls] if to_list else cls,
            Field(description=f"Provide your final answer {'as a list of' if to_list else 'here as'} {original_name}.")
        )

        return create_model(modified_name, **args)
    
    def unwrap_model(self):
        origin = getattr(self, self._original_name.lower())
        if isinstance(origin, list) and next(iter(origin))._is_wrapped:
            return [model.unwrap_model() for model in origin]
        return origin

In [144]:
class Joke(ModifiedBaseModel):
    setup: str
    punchline: str

In [156]:
model_0 = Joke.wrap_model(with_reasoning=True, to_list=False)
model_1 = (Joke
           .wrap_model(with_reasoning=True, to_list=False)
           .wrap_model(with_reasoning=True, to_list=True)
           )

In [157]:
test_model_1 = model_1(
    reasoning="These jokes are effective because they use wordplay and subvert expectations, making them entertaining and memorable.",
    jokewithreasoning=[
        {
            "reasoning": "This joke uses a classic setup with a scientific twist, leading to a clever punchline that plays on the dual meaning of 'make up'.",
            "joke": {
                "setup": "Why don't scientists trust atoms?",
                "punchline": "Because they make up everything!"
            }
        },
        {
            "reasoning": "This joke leverages the double meaning of 'outstanding in his field' to create humor through a pun.",
            "joke": {
                "setup": "Why did the scarecrow become a successful neurosurgeon?",
                "punchline": "Because he was outstanding in his field!"
            }
        }
    ]
)

test_model_0 = model_0(
    reasoning="This joke is humorous due to its use of a pun and the unexpected twist in the punchline.",
    joke={
        "setup": "I would tell you a construction joke,",
        "punchline": "but I'm still working on it."
    }
)


In [158]:
test_model_1.unwrap_model()

[Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!'),
 Joke(setup='Why did the scarecrow become a successful neurosurgeon?', punchline='Because he was outstanding in his field!')]

In [159]:
test_model_0.unwrap_model()

Joke(setup='I would tell you a construction joke,', punchline="but I'm still working on it.")